### Convert intersections described in Citations dataset into gis coordinates to be able to do location analysis with the Citations dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pyodbc
import pandas as pd
import seaborn as sns
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# databaseConnection = 'DRIVER={SQL Server}; SERVER=ServerName; Database=DatbaseName; UID=UserId; PWD=password;'
connection = pyodbc.connect('DRIVER={SQL Server};SERVER=WINDOWS-A6S01I2\SQLEXPRESS;DATABASE=City_of_Scottsdale;Trusted_Connection=yes;')
cursor = connection.cursor()

# SQL query to import all citation data
citations = '''
  select * from [dbo].[spd_PDCitations$]  
  '''
citations = pd.read_sql(citations, connection)
connection.close()

In [28]:
pd.set_option('display.max_columns', None)
citations.head()

,tcmainid,Citation #,Citation Date,Citation Time,Arizona Statute Code,Charge Description,Citation Type Code,Citation Type Description,Officer Badge #,Street Number,Street,City,State,Zip,District,Beat,Zone,Cited Person,Cited Person Age,Cited Person Race,Cited Person Sex,Cited Person Ethnicity,street#_street
0,621784.0,2065379.0,2018-06-23,204.0,13-1203A1,Assault-Intentional/Recklessly Injure,CR,Criminal,1489.0,1xxx,N Granite Reef Rd,Scottsdale,AZ,NaN,D1,2.0,412.0,"Hill, Julie",25.0,I,F,N,1xxx N Granite Reef Rd
1,621797.0,2063172.0,2018-06-23,928.0,13-3415A,Drug Paraphernalia-Possess/Use,CR,Criminal,1214.0,7xxx,N Hayden Rd,Scottsdale,AZ,NaN,D3,10.0,2504.0,"Mc Cuan, Michael",51.0,W,M,N,7xxx N Hayden Rd
2,621806.0,2056700.0,2018-06-23,1053.0,28-701A,Failure to Control Speed to Avoid A Collision,CV,Civil Traffic,1360.0,xxx,E Indian Bend Rd / N Scottsdale Rd,Scottsdale,AZ,NaN,D3,10.0,2501.0,"Glasscock, James",60.0,W,M,N,xxx E Indian Bend Rd / N Scottsdale Rd
3,621847.0,2050143.0,2018-06-23,2345.0,19-19A,Urinate/Defecate In Public,MC,Municipal Code,1345.0,4xxx,N Saddlebag Tr,Scottsdale,AZ,NaN,D2,6.0,904.0,"Still, Marcus",22.0,B,M,N,4xxx N Saddlebag Tr
4,621878.0,2063174.0,2018-06-24,927.0,28-3473A,Suspended/Revoked D.L.,CT,Criminal Traffic,1214.0,8xxx,E McDonald DR,Scottsdale,AZ,NaN,D2,8.0,2205.0,"Chacon, Kimberly",48.0,W,F,N,8xxx E McDonald DR


In [34]:
citations.dtypes

tcmainid                            float64
Citation #                          float64
Citation Date                datetime64[ns]
Citation Time                       float64
Arizona Statute Code                 object
Charge Description                   object
Citation Type Code                   object
Citation Type Description            object
Officer Badge #                     float64
Street Number                        object
Street                               object
City                                 object
State                                object
Zip                                 float64
District                             object
Beat                                float64
Zone                                float64
Cited Person                         object
Cited Person Age                    float64
Cited Person Race                    object
Cited Person Sex                     object
Cited Person Ethnicity               object
street#_street                  

In [27]:
citations['street#_street'] = citations['Street Number'] + " " + citations['Street']

In [30]:
citations.groupby('street#_street')['Citation #'].count().to_frame().reset_index().head()

,street#_street,Citation #
0,xxx 101st / Pine Valley,1
1,xxx 104th / Dynamite,1
2,xxx 115 / Shea,2
3,xxx 115th St / E Shea Bl,1
4,xxx 116th / Rio Verde,2


In [104]:
x = citations.groupby('Street')['Citation #'].count().to_frame().reset_index().sort_values(by='Street', ascending= False)
x.head(20)

,Street,Citation #
1726,Wells Fargo / E Stetson,1
1725,Scottsdale / Via Camello,1
1724,Scottsdale / Sundown,1
1723,Saddlebag,1
1722,Pirates Cove DR,1
1721,N Winfield Scott Pz,6
1720,N Williams DR,1
1719,N Whispering Ridge Wy,1
1718,N Wells Fargo Av,11
1717,N Via Paseo Del Sur,4


In [103]:
# filter for only cross streets
# exclude specific cross streets like L101 that are too ambiguous for google to provide latitude and longitude
include = ['/']
exclude = ['L101']

    
x[x['Street'].apply(lambda x: ('/' in str(x)) &
                  ('L101' not in str(x)))].sort_values(by='Citation #', ascending= False)
                  

,Street,Citation #
1694,N Scottsdale Rd / E Thomas Rd,239
1546,N Hayden Rd / E Indian School Rd,234
886,N 64th St / E Cholla St,216
1685,N Scottsdale Rd / E Shea Bl,216
188,E Camelback Rd / N Saddlebag Tr,211
189,E Camelback Rd / N Scottsdale Rd,205
468,E McDowell Rd / N Scottsdale Rd,202
315,E Frank Lloyd Wright Bl / N Hayden Rd,156
1564,N Hayden Rd / E McDowell Rd,153
318,E Frank Lloyd Wright Bl / N Scottsdale Rd,152
